# Homework 6
Dr. Kerby <br>
Data Science and Applied Machine Learning <br>
Student's name: Ziming Wang

For a description of the HousePrices.csv dataset, visit:</br>
https://gitlab.com/CEADS/DrKerby/python/blob/master/HousePrices_description.txt

In [335]:
# Just execute this cell
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

fulldf = pd.read_csv('https://raw.githubusercontent.com/LGKerby/Python/master/HousePrices.csv')
fulldf.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


**Question 1** (10 points)<br/>
Explore the dataset using at least three different methods 

In [336]:
# How many houses' utilities are Allpub.
len(fulldf.groupby('Utilities').get_group('AllPub'))

1459

In [337]:
# How many houses are bulit over 2000
len(fulldf[fulldf['YearBuilt'] >= 2000])

388

In [338]:
# What is the average price of a house under 2000?
fulldf[fulldf['YearBuilt'] < 2000]['SalePrice'].mean()

158655.36473880598

There are 81 columns in our dataset. One column is the ID (which is not valuable data but an after-label), and another column is the Sale Price, which is what we will try to predict with machine learning. Therefore, we have 79 possible factors to use to create our model. Of these, we will keep the following 16:<br/>
*MSZoning, LotArea, Neighborhood, BldgType, OverallQual, OverallCond, YearBuilt, TotalBsmtSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, GarageCars, SaleCondition*

**Question 2** (10 points)<br/>
Create a DataFrame `housedf` which keeps only these 16 columns from `fulldf`. Print `house.head()` and `house.shape` to show you did it right.

In [339]:
housedf = fulldf[['MSZoning','LotArea','Neighborhood','BldgType','OverallQual','OverallCond','YearBuilt','TotalBsmtSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','GarageCars','SaleCondition']]
housedf.head()

,MSZoning,LotArea,Neighborhood,BldgType,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,GarageCars,SaleCondition
0,RL,8450,CollgCr,1Fam,7,5,2003,856,1710,1,0,2,1,3,2,Normal
1,RL,9600,Veenker,1Fam,6,8,1976,1262,1262,0,1,2,0,3,2,Normal
2,RL,11250,CollgCr,1Fam,7,5,2001,920,1786,1,0,2,1,3,2,Normal
3,RL,9550,Crawfor,1Fam,7,5,1915,756,1717,1,0,1,0,3,3,Abnorml
4,RL,14260,NoRidge,1Fam,8,5,2000,1145,2198,1,0,2,1,4,3,Normal


In [340]:
housedf.shape

(1460, 16)

**Question 3** (10 points) <br/>
Place the `SalePrice` column (from `fulldf`) into a variable called `answers`. Show `answers.head()`.

In [341]:
answers = fulldf['SalePrice']
answers.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

**Question 4** (10 points) <br/>
12 of our factors in `housedf` are numeric. 4 are categorical. The 4 categorical factors (MSZoning, Neighborhood, BldgType, and SaleCondition) need to be converted using `pandas` `get_dummies`. Do this and assign the result into a new DataFrame called `df`. Print `df.head()` to make sure you completed it correctly. `df` should now have 53 columns. In a separate cell, print `df.columns` so you can see how `get_dummies` handled the categorical factors, and what our 53 factors now are.

*Note:* OverallQual and OverallCond are ordinal. They are rankings from 1 to 10 on the overall quality and condition of the home, respectively. As a ranking of 10 (very excellent) should contribute more to the value of a home than a ranking of 1 (very poor), it is an acceptable approximation to keep these as numeric factors rather than use `get_dummies` and add another 18 factors to our model. However, this means the relationship within OverallQual and OverallCond will be forced to be linear among the rankings and may not represent its real impact on value. You are welcome to also analyze converting these factors using `get_dummies` if you are curious about it, but you do not have to.

In [342]:
df = pd.get_dummies(housedf)
df.head()

,LotArea,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,GarageCars,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,8450,7,5,2003,856,1710,1,0,2,1,3,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,9600,6,8,1976,1262,1262,0,1,2,0,3,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0
2,11250,7,5,2001,920,1786,1,0,2,1,3,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,9550,7,5,1915,756,1717,1,0,1,0,3,3,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,14260,8,5,2000,1145,2198,1,0,2,1,4,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [343]:
df.columns

Index(['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'TotalBsmtSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'GarageCars', 'MSZoning_C (all)', 'MSZoning_FV',
       'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Neighborhood_Blmngtn',
       'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor',
       'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes',
       'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge',
       'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU',
       'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst',
       'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker',
       'BldgType_1Fam', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs

**Question 5** (10 points) <br/>
Now split your `df` and `answers` dataset into a training and testing set, using `train_test_split`. Call the results `Xtrain`, `Xtest`, `ytrain`, and `ytest`. Print `head()` and `shape` for each to ensure the data was split correctly.

In [344]:
Xtrain, Xtest, ytrain, ytest = train_test_split(df, answers)

In [345]:
print(Xtrain.shape)
Xtrain.head()

(1095, 53)


,LotArea,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,GarageCars,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1178,7681,5,6,1921,731,1343,0,0,1,1,3,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
840,12155,6,8,1925,672,1482,0,0,2,0,4,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1238,13072,6,5,2005,1141,1141,0,0,1,1,3,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
432,1920,5,5,1971,765,1365,1,0,1,1,2,2,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
797,7677,5,5,1953,773,773,0,0,1,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [346]:
print(ytrain.shape)
ytrain.head()

(1095,)


1178    154900
840     140000
1238    142500
432     122500
797     110000
Name: SalePrice, dtype: int64

In [347]:
print(Xtest.shape)
Xtest.head()

(365, 53)


,LotArea,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,GarageCars,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
652,8750,7,5,1996,880,1716,0,0,2,1,2,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1036,12898,9,5,2007,1620,1620,1,0,2,0,2,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0
215,10011,5,6,1957,1070,1236,0,1,1,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
964,11316,7,5,2002,817,1894,1,0,2,1,4,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0
548,8235,5,7,1955,825,825,1,0,1,0,2,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [348]:
print(ytest.shape)
ytest.head()

(365,)


652     191000
1036    315500
215     134450
964     214900
548     125000
Name: SalePrice, dtype: int64

**Question 6** (25 points) <br/>
**a)** Now create your linear regression model, using `LinearRegression().fit` on your training data. Assign it to `regmodel`. <br/>
**b)** Calculate the predictions the model gives for your test dataset, assigning it to the variable `predictions`. <br/>
**c)** Print the $RMSE$ and $R^2$ values (for both the training and testing data). <br/>
**d)** Copy your `train_test_split` command from Question 5 to the top of the next cell, so that you can run that cell and do one complete random sampling of the train/test data, linear model generation, model statistics, and model equation.  

In [349]:
# a)
regmodel = LinearRegression().fit(Xtrain, ytrain)

In [350]:
# b)
predictions = regmodel.predict(Xtest)
predictions

array([212758.77609701, 278953.27719022, 145832.93983745, 230566.10919907,
       126477.03686998, 282399.51016174, 328902.87529415, 225981.15717877,
       135393.93714077, 237801.79549615, 127952.30341586, 192948.01984134,
       127288.55616582,  11433.7712653 , 114521.38600401, 252129.76333936,
       117056.94737919, 283385.46382936, 331030.32356495, 214576.29158183,
       203594.10350657, 131162.97549935, 338239.20305452, 124527.82703146,
       157831.08026541, 132547.42984871, 187718.86973627, 244217.30718843,
       216479.88930013, 102374.08618422, 204914.96868354, 204628.8495067 ,
       157960.29524077, 174778.24516486, 164370.48838739,  68384.55894483,
       237340.43497201, 153328.37092891, 163742.86615768, 139937.09851297,
       114292.84521176,  75924.1717275 ,  66760.03051443, 147797.95259253,
       276281.68017764, 129447.61788766, 184392.27333601, 172392.99839216,
       133253.51835886, 207195.03576788, 227188.77259175, 184041.59563657,
       244773.14629456, 1

In [351]:
# c)
# For testing data
rmse_test = math.sqrt(mean_squared_error(ytest, predictions))
R2_test = regmodel.score(Xtest, ytest)
print(rmse_test)
print(R2_test)

31683.98660556105
0.8290968572210715


In [352]:
# For training data
predictions_train = regmodel.predict(Xtrain)
rmse_train = math.sqrt(mean_squared_error(ytrain, predictions_train))
R2_train = regmodel.score(Xtrain, ytrain)
print(rmse_train)
print(R2_train)

31440.46637903175
0.846684924791286


In [353]:
# d)
# one complete random sampling of the train/test data
Xtrain, Xtest, ytrain, ytest = train_test_split(df, answers)
# linear model generation
regmodel = LinearRegression().fit(Xtrain, ytrain)
# model statistics
predictions = regmodel.predict(Xtest)
predictions_train = regmodel.predict(Xtrain)

rmse_test = math.sqrt(mean_squared_error(ytest, predictions))
R2_test = regmodel.score(Xtest, ytest)
rmse_train = math.sqrt(mean_squared_error(ytrain, predictions_train))
R2_train = regmodel.score(Xtrain, ytrain)
print(rmse_test)
print(R2_test)
print(rmse_train)
print(R2_train)

# model equation
regmodel.coef_
regmodel.intercept_

equation = 'y = '
equation += '({:.4f}) + '.format(regmodel.intercept_)
for i in np.arange(regmodel.coef_.size):
    equation += '({:.4f})*{} + '.format(regmodel.coef_[i],df.columns[i])
equation

36719.1769969404
0.8352188134742347
29840.23220883673
0.8432617083019517


'y = (-707597.9855) + (0.3872)*LotArea + (13088.7125)*OverallQual + (5920.0850)*OverallCond + (334.5907)*YearBuilt + (12.1438)*TotalBsmtSF + (42.3586)*GrLivArea + (14545.0779)*BsmtFullBath + (8951.2286)*BsmtHalfBath + (6613.1847)*FullBath + (2654.3311)*HalfBath + (-4200.2928)*BedroomAbvGr + (11662.5901)*GarageCars + (-14972.4084)*MSZoning_C (all) + (-569.2936)*MSZoning_FV + (15678.9716)*MSZoning_RH + (1842.9780)*MSZoning_RL + (-1980.2476)*MSZoning_RM + (1653.2531)*Neighborhood_Blmngtn + (-4010.0185)*Neighborhood_Blueste + (7015.3662)*Neighborhood_BrDale + (-11191.3519)*Neighborhood_BrkSide + (-4457.2608)*Neighborhood_ClearCr + (-10717.3241)*Neighborhood_CollgCr + (17213.7162)*Neighborhood_Crawfor + (-23339.3101)*Neighborhood_Edwards + (-19307.3115)*Neighborhood_Gilbert + (-13660.4197)*Neighborhood_IDOTRR + (10736.6575)*Neighborhood_MeadowV + (-19979.3215)*Neighborhood_Mitchel + (-15353.3176)*Neighborhood_NAmes + (6266.7915)*Neighborhood_NPkVill + (-20222.0611)*Neighborhood_NWAmes + (33

**Question 7** (25 points) <br/>
Run the `train_test_split` and `LinearRegression().fit` several times, noting the RMSE and R^2 train and test results, along with the regression equation found by the model. Answer the following questions.

What appears to be the rough average for RMSE? <br/>
What appears to be the rough averages for R^2? <br/>

Name three things that make sense in your model.<br/>
Name two things that don't make sense in your model.<br/>

Based on your model, what would you expect the three wealthiest neighborhoods to be? <br/>
What would you expect the three poorest neighborhoods to be? 

Are you concerned you overfit your model? 

In [354]:
RMSE_SUM = 0
R2_Sum = 0
for a in range(1000):
  Xtrain, Xtest, ytrain, ytest = train_test_split(df, answers)
  regmodel = LinearRegression().fit(Xtrain, ytrain)
  predictions_Ave = regmodel.predict(Xtest)
  RMSE_SUM = RMSE_SUM + math.sqrt(mean_squared_error(ytest, predictions_Ave))
  R2_SUM = R2_SUM + regmodel.score(Xtest, ytest)
EMSE_ave = RMSE_SUM / 1000
R2_ave = R2_SUM / 1000
print(EMSE_ave)
R2_ave

33373.04350530607


0.818589180855767

Put your answers here.<br/>
The average for RMSE is 33601.<br/>
The average for R^2 is 0.814.<br/>
<br/>
Make sense: BsmtFullBath, OverallQual, and OverallCond.<br/>
Not make sense: Neighborhood_Gilbert and Neighborhood_Edwards.<br/>
<br/>
Three wealthies neighborhoods:Neighborhood_NoRidge, Neighborhood_NridgHt, and Neighborhood_StoneBr.<br/>
Three poorest neighborhoods:Neighborhood_Edwards, Neighborhood_Gilbert, and Neighborhood_IDOTRR.<br/>
<br/>
RMSE for the testing data is higher than RMSE for the training data. So my model is overfit.<br/>

*One final note:* If we had more than just ~1500 homes, we could create machine learning models using all of the factors available in `fulldf`. 

**Graduate Students Only:** \\
Try adding and/or subtracting features, or creating your own, and building a new linear regression model with those features. Discuss advantages and disadvantages. Which is better?